In [1]:
%%capture

!pip install ipython-autotime
%load_ext autotime

time: 410 µs (started: 2023-08-14 09:22:25 +00:00)


In [2]:
%%capture
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes>=0.41.1 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq huggingface-hub==0.16.4 --progress-bar off
!pip install -qqq sentencepiece==0.1.99 --progress-bar off

time: 1min 5s (started: 2023-08-14 09:22:25 +00:00)


In [3]:
%%capture
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git

time: 49.1 s (started: 2023-08-14 09:26:07 +00:00)


In [4]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)

time: 9.85 s (started: 2023-08-14 09:26:56 +00:00)


In [5]:

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)


time: 288 µs (started: 2023-08-14 09:27:06 +00:00)


In [6]:
import torch
from huggingface_hub import notebook_login
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
notebook_login()

time: 49.2 ms (started: 2023-08-14 09:27:06 +00:00)


## Dataset

In [9]:
# importing the zipfile module
from zipfile import ZipFile

zip_path = "/content/archive.zip"

# loading the temp.zip and creating a zip object
with ZipFile(zip_path, 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall()

time: 192 ms (started: 2023-08-14 09:29:37 +00:00)


In [10]:
import json

file_path = '/content/train-v1.1.json'
with open(file_path, 'r') as json_file:
        data = json.load(json_file)

time: 750 ms (started: 2023-08-14 09:29:42 +00:00)


In [11]:
data['data'][0]['paragraphs'][1]['qas']

[{'answers': [{'answer_start': 248, 'text': 'September 1876'}],
  'question': 'When did the Scholastic Magazine of Notre dame begin publishing?',
  'id': '5733bf84d058e614000b61be'},
 {'answers': [{'answer_start': 441, 'text': 'twice'}],
  'question': "How often is Notre Dame's the Juggler published?",
  'id': '5733bf84d058e614000b61bf'},
 {'answers': [{'answer_start': 598, 'text': 'The Observer'}],
  'question': 'What is the daily student paper at Notre Dame called?',
  'id': '5733bf84d058e614000b61c0'},
 {'answers': [{'answer_start': 126, 'text': 'three'}],
  'question': 'How many student news papers are found at Notre Dame?',
  'id': '5733bf84d058e614000b61bd'},
 {'answers': [{'answer_start': 908, 'text': '1987'}],
  'question': 'In what year did the student paper Common Sense begin publication at Notre Dame?',
  'id': '5733bf84d058e614000b61c1'}]

time: 4.32 ms (started: 2023-08-14 09:29:47 +00:00)


In [12]:
#general function for multiple files
import os
qa_list = []
for i in data['data'][0]['paragraphs']:
  for j in i['qas']:
    answer = j['answers'][0]['text']
    question = j['question']

    qa_list.append({"question":question, "answer": answer})

data = {"questions":qa_list}



time: 1.06 ms (started: 2023-08-14 09:29:50 +00:00)


In [13]:
import json
with open("squad_dataset.json","w") as f:
  json.dump(data['questions'],f)

time: 3.49 ms (started: 2023-08-14 09:29:51 +00:00)


In [15]:
%%capture
!pip install datasets

time: 5.93 s (started: 2023-08-14 09:30:02 +00:00)


In [16]:
from datasets import load_dataset
import json
data = load_dataset("json", data_files = "squad_dataset.json",  split="train")
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'answer'],
    num_rows: 269
})

time: 1.71 s (started: 2023-08-14 09:30:14 +00:00)


In [17]:
from datasets import Dataset

data_with_text = Dataset.from_dict(
    {
        'text': [f"{q} {a}" for q, a in zip(data['question'], data['answer'])],
        'question': data['question'],
        'answer': data['answer'],
    }
)

time: 6.33 ms (started: 2023-08-14 09:30:18 +00:00)


## Loading the Model

In [18]:
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

time: 447 µs (started: 2023-08-14 09:30:22 +00:00)


In [19]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = (0)
tokenizer.padding_side = "left"

time: 2.33 s (started: 2023-08-14 09:30:26 +00:00)


In [20]:
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

time: 2min 40s (started: 2023-08-14 09:30:31 +00:00)


## Finetuning

In [21]:
def generate_prompt(data_point):
  return f"""
  <human>: {data_point['question']}
  <assistant>: {data_point['answer']}
  """.strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding = True, truncation = True)
  return tokenized_full_prompt

data = data.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


time: 443 ms (started: 2023-08-14 09:33:11 +00:00)


In [22]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 1
MICRO_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300
OUTPUT_DIR = "experiments"

time: 659 µs (started: 2023-08-14 09:33:23 +00:00)


In [23]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=4, #LORA_R,
    lora_alpha=16, #LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=0.05, #LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 2,097,152 || all params: 6,740,512,768 || trainable%: 0.03111264783824826
time: 11.2 s (started: 2023-08-14 09:33:24 +00:00)


In [24]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.32.0.dev0"
}

time: 407 ms (started: 2023-08-14 09:33:35 +00:00)


In [25]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

time: 52.6 ms (started: 2023-08-14 09:33:36 +00:00)


In [26]:
%%capture
!pip install trl

time: 9.15 s (started: 2023-08-14 09:33:42 +00:00)


In [27]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=data_with_text,
    peft_config=config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

time: 400 ms (started: 2023-08-14 09:33:51 +00:00)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [28]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

time: 8.79 ms (started: 2023-08-14 09:33:58 +00:00)


In [29]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,3.280000
20,2.772800
30,2.257400
40,2.004000
50,1.961000
60,1.772600
70,1.662500
80,1.338900
90,1.243200
100,1.161900


TrainOutput(global_step=100, training_loss=1.9454410362243653, metrics={'train_runtime': 663.0867, 'train_samples_per_second': 2.413, 'train_steps_per_second': 0.151, 'total_flos': 1434539462049792.0, 'train_loss': 1.9454410362243653, 'epoch': 5.88})

time: 11min 3s (started: 2023-08-14 09:34:01 +00:00)


In [30]:
logs = trainer.state.log_history
logs[-1]['train_loss']

1.9454410362243653

time: 3.66 ms (started: 2023-08-14 09:47:42 +00:00)


In [31]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

time: 33.6 ms (started: 2023-08-14 09:47:45 +00:00)


In [32]:
model_to_save.push_to_hub("suchetajjw47/fine_tuned_llama2-squad", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:831: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/suchetajjw47/fine_tuned_llama2-squad/commit/711e0d35bb7860109fb8605012f986172fadcab0', commit_message='Upload model', commit_description='', oid='711e0d35bb7860109fb8605012f986172fadcab0', pr_url=None, pr_revision=None, pr_num=None)

time: 4.85 s (started: 2023-08-14 09:47:48 +00:00)


# Testing the fine tuned model

In [33]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

time: 77.1 ms (started: 2023-08-14 09:47:59 +00:00)


In [34]:
device = "cuda:0"
text = "What is the Grotto at Notre Dame?"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


What is the Grotto at Notre Dame?
The Grotto at Notre Dame is a replica of the grotto at Lourdes, France, and is the most visited spot on campus. It is located on the north side of the Main Building (aka the Administration Building)
time: 16 s (started: 2023-08-14 09:48:00 +00:00)


In [35]:
text = "Where did Notre Dame students and the KKK have their encounter?"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Where did Notre Dame students and the KKK have their encounter?
On October 19, 1924, the campus of Notre Dame University was the site of a clash between students and members of the Ku Klux Klan. The incident occurred at the conclusion of a public address by
time: 15.3 s (started: 2023-08-14 09:48:16 +00:00)
